In [1]:
import os
import pandas as pd
import geopandas as gpd

In [2]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [3]:
### Read .csv file
df = pd.read_csv(cwd + "\\Site.csv")
df.head()

,Number,Site,Serial Number (FLOX),Country,Location,Latitude,Longitude,Priority,Plant functional type (PFT),Reference network,Number of components,Reduce ROI size (600 x 600 m),Move ROI,Comments,PRISMA Availability (900m Buffer ROI Check),PRISMA Earliest Date,PRISMA Latest Date (Before 2024-06-26),Tomi,S2 Cloud
0,1,ATGE,NaN,NaN,NaN,52.466778,12.959778,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,2,2022-05-06,2023-08-14,Y,NaN
1,2,ATLAS-Mohammed V,NaN,Morocco,ATLAS-Mohammed V,33.406152,-5.103319,Low,NaN,NaN,NaN,Yes,Yes,"mixed pixel, complex topography",2,2023-11-23,2023-11-23,NaN,NaN
2,3,AU-MIEMING,JB-042-GW,Austria,Austria,47.316700,10.970300,NaN,NaN,FLOX,NaN,No,No,topography?,0,NaN,NaN,Duplicate,NaN
3,4,BASP,NaN,NaN,NaN,39.049139,-2.075917,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,27,2022-05-09,2024-06-18,Y,NaN
4,5,BE-BRASCHAAT,JB-021-SW,Belgium,Brasschaat,51.307600,4.519900,NaN,NaN,FLOX,NaN,No,No,Check the site with Sentinal 3,7,2020-09-12,2022-06-01,NaN,NaN


In [13]:
for i in range(df.shape[0]):
    if not os.path.isdir(cwd_Images_Processed + "\\" + df['Site'][i]):
        os.mkdir(cwd_Images_Processed + "\\" + df['Site'][i])
    temp_GDF_4326 = gpd.GeoDataFrame(
        pd.DataFrame({
            "Site": [df['Site'][i]],
        }),
        geometry = gpd.points_from_xy([df['Longitude'][i]], [df['Latitude'][i]]),
        crs = "EPSG:4326"
    )
    if df['Site'][i] == 'INDONESIA':    #
        temp_GDF_32632 = temp_GDF_4326.to_crs("EPSG:23830")
        print("INDONESIA!")
    else:
        temp_GDF_32632 = temp_GDF_4326.to_crs("EPSG:32632")
    temp_GDF_32632_Buffer = temp_GDF_32632.copy().buffer(900, cap_style = 3)
    temp_GDF_4326_Buffer = temp_GDF_32632_Buffer.to_crs("EPSG:4326")
    temp_GDF_4326_Buffer.to_file(cwd_Images_Processed + "\\" + df['Site'][i] + "\\buffer_900m_4326.shp")
    temp_GDF_32632_Buffer = temp_GDF_32632.copy().buffer(1800, cap_style = 3)
    temp_GDF_4326_Buffer = temp_GDF_32632_Buffer.to_crs("EPSG:4326")
    temp_GDF_4326_Buffer.to_file(cwd_Images_Processed + "\\" + df['Site'][i] + "\\buffer_1800m_4326.json", driver = "GeoJSON")

INDONASIA!


# Concatenation

In [4]:
# Concatenate .csv results of Roman Metrics
for i in range(df.shape[0]):
    if os.path.exists(cwd_Images_Processed + "\\" + df['Site'][i] + "\\Result_New_Variogram.csv"):
        temp_csv = pd.read_csv(cwd_Images_Processed + "\\" + df['Site'][i] + "\\Result_New_Variogram.csv")
        if i == 0:
            final_csv_Variogram = temp_csv
        else:
            final_csv_Variogram = pd.concat((final_csv_Variogram,temp_csv), ignore_index= True)
final_csv_Variogram.head(10)

,Site,Distance,Range,Sill,Nugget,STD,CV
0,ATGE,30m,20.000000,99.392275,3.570080e+01,10.292980,0.256794
1,ATGE,100m,70.000000,267.887789,4.884311e-14,15.866641,0.478179
2,ATGE,300m,104.664413,449.836840,6.333086e+01,20.748732,0.428943
3,ATGE,900m,345.097180,438.814722,1.064037e+02,20.035755,0.502820
4,ATGE,1200m,327.228777,419.066655,8.059260e+01,19.909969,0.527853
5,ATLAS-Mohammed V,30m,20.000000,17.356565,8.395671e-01,1.490574,0.055660
6,ATLAS-Mohammed V,100m,30.284278,18.800881,1.837208e-11,4.409100,0.170407
7,ATLAS-Mohammed V,300m,98.388819,39.292309,1.650852e+01,6.381924,0.236488
8,ATLAS-Mohammed V,600m,420.000000,69.804207,2.620777e+01,7.898373,0.242353
9,ATLAS-Mohammed V,900m,630.000000,131.031061,1.541876e+00,11.160982,0.461659


In [5]:
final_csv_Variogram.to_csv(cwd + "\\Result - Variogram.csv", index = False)

In [6]:
# Concatenate .csv results of Roman Metrics
for i in range(df.shape[0]):
    if os.path.exists(cwd_Images_Processed + "\\" + df['Site'][i] + "\\Result_New_Roman.csv"):
        temp_csv = pd.read_csv(cwd_Images_Processed + "\\" + df['Site'][i] + "\\Result_New_Roman.csv")
        if i == 0:
            final_csv_Roman = temp_csv
        else:
            final_csv_Roman = pd.concat((final_csv_Roman,temp_csv), ignore_index= True)
final_csv_Roman.head(10)

,Site,Internal Distance,External Distance,Rcv,Rse,Rst,Rsv,RAW Score,ST Score
0,ATGE,30m,100m,0.862114,0.784802,0.990000,8.646682,0.579970,0.233405
1,ATGE,30m,300m,0.670379,0.788821,0.716754,11.393105,0.745846,0.198063
2,ATGE,100m,300m,-0.102966,0.923035,-0.137310,0.284701,4.855959,0.910724
3,ATGE,300m,900m,0.172231,0.954540,-0.136330,1.906935,2.903070,0.590654
4,ATGE,300m,1200m,0.230590,0.954330,-0.068328,1.938959,2.168347,0.588135
5,ATLAS-Mohammed V,30m,100m,2.061553,0.756370,0.990000,29.990659,0.242536,0.084959
6,ATLAS-Mohammed V,30m,300m,3.248775,0.788388,0.126361,57.381840,0.153904,0.047527
7,ATLAS-Mohammed V,100m,300m,0.387784,0.851292,-0.433990,0.883853,1.289376,0.704307
8,ATLAS-Mohammed V,300m,900m,0.952147,0.953194,0.750471,9.912785,0.525129,0.207254
9,ATLAS-Mohammed V,300m,1200m,1.152241,0.953432,0.766752,13.723637,0.433937,0.162137


In [7]:
final_csv_Roman.to_csv(cwd + "\\Result - Roman.csv", index = False)

In [17]:
site_Temp = "FR-Pue"

In [18]:
final_csv_Variogram.loc[final_csv_Variogram['Site'] == site_Temp,:]

,Site,Distance,Range,Sill,Nugget,STD,CV
186,FR-Pue,30m,20.000000,14.103029,3.402373e+00,3.415670,0.068917
187,FR-Pue,100m,70.000000,20.634004,3.097274e-19,4.802768,0.096683
188,FR-Pue,300m,138.298038,40.060909,1.066629e+01,6.092970,0.129308
189,FR-Pue,600m,373.885472,79.575313,2.628870e+01,8.944910,0.209535
190,FR-Pue,900m,630.000000,126.926265,2.017268e+01,10.748985,0.275092
191,FR-Pue,1200m,824.080720,129.539704,4.274351e+01,10.650192,0.281218


In [19]:
final_csv_Roman.loc[final_csv_Roman['Site'] == site_Temp,:]

,Site,Internal Distance,External Distance,Rcv,Rse,Rst,Rsv,RAW Score,ST Score
186,FR-Pue,30m,100m,0.402882,0.784802,0.990000,13.362622,1.241057,0.175337
187,FR-Pue,30m,300m,0.876280,0.790236,0.421882,19.820865,0.570594,0.127715
188,FR-Pue,100m,300m,0.337447,0.928102,-0.285487,0.449656,1.481715,0.777827
189,FR-Pue,300m,900m,1.127422,0.966091,0.191487,4.221658,0.443490,0.355499
190,FR-Pue,300m,1200m,1.174797,0.966411,-0.045674,4.441322,0.425605,0.350425
191,FR-Pue,300m,600m,0.620434,0.964711,-0.062092,1.467262,0.805887,0.594775
